In [1]:
!pip install transformers conllu datasets

In [2]:
from datasets import load_dataset , get_dataset_config_names

In [3]:
# Fetching en_ewts train, validation splits
train_dataset = load_dataset('universal_dependencies', 'en_ewt', split = 'train')
validation_dataset = load_dataset('universal_dependencies', 'en_ewt', split = 'validation')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2002 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2077 [00:00<?, ? examples/s]

In [11]:
# Find total pos

total_pos = set()
total_sentences = train_dataset.num_rows

for i in range(0,total_sentences):
  for pos in train_dataset[i]['xpos']:
    total_pos.add(pos)

total_pos

{'$',
 "''",
 ',',
 '-LRB-',
 '-RRB-',
 '.',
 ':',
 'ADD',
 'AFX',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'GW',
 'HYPH',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NFP',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 None,
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 'XX',
 '``'}

Preprocessing data

UD dataset has arond 50 xpos.For the sake of simplicity I am prepocessing data to have only 6 labels.

1. NOUN - NN
2. ADJECTIVE - JJ
3. VERB - VB
4. INTERJECTION - IN
5. ADVERB - ADV
6. OTHERS - OTHERS

In [6]:
total_sentences = train_dataset.num_rows
train_X = [row['tokens'] for row in train_dataset]
train_Y = [row['xpos'] for row in train_dataset]

for i in range(0,len(train_Y)):
  for j in range(0, len(train_Y[i])):
    pos = train_Y[i][j]
    if(pos in ['NN', 'NNP', 'NNS']):
      pos = 'NN'
    elif(pos in ['VB', 'VBD', 'VBG']):
      pos = 'VB'
    elif(pos != 'JJ' and pos != 'IN' and pos != 'RB'):
      pos = 'OTHERS'
    train_Y[i][j] = pos

In [7]:
# Find total pos

total_pos = set()
total_sentences = train_dataset.num_rows

for i in range(0,total_sentences):
  for pos in train_Y[i]:
    total_pos.add(pos)

total_pos= sorted(total_pos)
total_pos

['IN', 'JJ', 'NN', 'OTHERS', 'RB', 'VB']

Featuraization

We define some static features to convert the input token as a feature
1. If the given token is a 'NOUN' then the corresponding feature is true else false {0, 1}
2. Similarly for other pos tags - 4 others
3. Transition features : 1 if given token is a NOUN and previous token is VERB.
4. Similarly for other combinations - 24 others


In [9]:
def featurize_word(word, pos, prev_pos):
  feature = [0 for i in range(42)]

  for i in range(0, len(total_pos)):
    if(pos == total_pos[i]):
      feature[i]=1

  # transition features
  for i in range(0,len(total_pos)):
    for j in range(0,len(total_pos)):
      if(total_pos[i] == pos and total_pos[j] == prev_pos):
        feature[len(total_pos)-1+(i+1)*(j+1)] = 1

  return feature

def featurize_sentence(sentence,pos):
  # Sentence is a list of tokens
  sentence_feature = []
  for i in range(0,len(sentence)):
    if(i == 0):
      sentence_feature.append(featurize_word(sentence[i],pos[i],'OTHER'))
    else:
      sentence_feature.append(featurize_word(sentence[i],pos[i],pos[i-1]))
  return sentence_feature

def featurize_dataset(train_X, train_Y):
  dataset_features = []
  for i in range(0, len(train_X)):
    dataset_features.append(featurize_sentence(train_X[i], train_Y[i]))
  return dataset_features

In [10]:
features = featurize_dataset(train_X, train_Y)

len(features)

12543